<a href="https://colab.research.google.com/github/DAHYUN-HAN/Machine_Learning_Titanic/blob/master/submit_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [2]:
#colab에서 g란 이름으로 구글드라이브의 root를 MOUNT
from google.colab import drive
drive.mount('/content/g')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/g


In [0]:
!mkdir -p ~/.kaggle
!cp "/content/g/My Drive/kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 5.90MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.4MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 51.3MB/s]


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [0]:
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [7]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [0]:
del df_train['PassengerId']
del df_train['Name']
del df_train['Ticket']
del df_train['Cabin']
df_test['PassengerId']
del df_test['Name']
del df_test['Ticket']
del df_test['Cabin']

In [0]:
df_train.dropna(thresh=int(len(df_train)*0.5), axis=1)
df_train['Embarked']=df_train['Embarked'].fillna('S')
df_train['Age']=df_train['Age'].fillna(df_train['Age'].mean())

In [0]:
df_test.dropna(thresh=int(len(df_test)*0.5), axis=1)
df_test['Fare']=df_test['Fare'].fillna(df_test['Fare'].mean())
df_test['Age']=df_test['Age'].fillna(df_test['Age'].mean())

In [0]:
df_train['Sex']=df_train['Sex'].astype('category')
df_train['Pclass']=df_train['Pclass'].astype('category')
df_train['Embarked']=df_train['Embarked'].astype('category')
df_train = pd.get_dummies(df_train)

df_test['Sex']=df_test['Sex'].astype('category')
df_test['Pclass']=df_test['Pclass'].astype('category')
df_test['Embarked']=df_test['Embarked'].astype('category')
df_test = pd.get_dummies(df_test)

In [12]:
x_data = df_train.iloc[:,1:]
y_data = df_train.iloc[:,0]

x_data = x_data.values
y_data = y_data.values
print(x_data)
print(y_data)

[[22.          1.          0.         ...  0.          0.
   1.        ]
 [38.          1.          0.         ...  1.          0.
   0.        ]
 [26.          0.          0.         ...  0.          0.
   1.        ]
 ...
 [29.69911765  1.          2.         ...  0.          0.
   1.        ]
 [26.          0.          0.         ...  1.          0.
   0.        ]
 [32.          0.          0.         ...  0.          1.
   0.        ]]
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 

In [44]:
x2_data = df_test.iloc[:,1:]

x2_data = x2_data.values
print(x2_data)

[[34.5         0.          0.         ...  0.          1.
   0.        ]
 [47.          1.          0.         ...  0.          0.
   1.        ]
 [62.          0.          0.         ...  0.          1.
   0.        ]
 ...
 [38.5         0.          0.         ...  0.          0.
   1.        ]
 [30.27259036  0.          0.         ...  0.          0.
   1.        ]
 [30.27259036  1.          1.         ...  1.          0.
   0.        ]]


In [14]:
estimator = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)
estimator.fit(x_data, y_data)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [0]:
y_predict = estimator.predict(x2_data)

In [46]:
df_test

,PassengerId,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,34.50000,0,0,7.8292,0,0,1,0,1,0,1,0
1,893,47.00000,1,0,7.0000,0,0,1,1,0,0,0,1
2,894,62.00000,0,0,9.6875,0,1,0,0,1,0,1,0
3,895,27.00000,0,0,8.6625,0,0,1,0,1,0,0,1
4,896,22.00000,1,1,12.2875,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,30.27259,0,0,8.0500,0,0,1,0,1,0,0,1
414,1306,39.00000,0,0,108.9000,1,0,0,1,0,1,0,0
415,1307,38.50000,0,0,7.2500,0,0,1,0,1,0,0,1
416,1308,30.27259,0,0,8.0500,0,0,1,0,1,0,0,1


In [0]:
submission=pd.DataFrame({"PassengerId":df_test["PassengerId"], "Survived":y_predict})

In [0]:
submission.to_csv('submission.csv', index=False)